In [1]:
# Want B to C   removing one of the pairs 

#The Goal is to take a pdb file and identify all the WC base pairs and the hydrogen bonded atoms(?)
# Then create an input file in Plumed's format to define the distances between those hydrogen bonded atoms so
# that a constraints/restraint function can be added later

# This is specifically for CD44 SS_B right now though can be reworked for other uses

# Running Plumed with these intial constraints/restraints will then allow me to reduce the phase space and 
# then be able to reduce computational run time

# Inputs: .pdb file
# Return: some plumed input file (?)

# Made by Isabel Dengos (ijdengos@gmail.com)
# February 2022
# Chen Lab, RNA Institute, UAlbany

In [2]:
# Import all packages needed
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import numpy as np
import sys

In [3]:
# Import all files needed
pdb = PDBFile("/network/rit/home/ID653677/ChenRNALab/idengos/CD44/SS_B/DistAndHHeatGromacs/conf.pdb")

In [4]:
#beginig of Plumed output build
k = 0

lines = ['#SETTINGS MOLFILE=/network/rit/home/ID653677/ChenRNALab/idengos/CD44/SS_B/DistAndHHeatGromacs/conf.pdb', 
         '# vim:ft=plumed', 'MOLINFO STRUCTURE=conf.pdb']

with open('plumed2.dat', 'w') as f:
    for line in lines:
        f.write(line)
        f.write('\n')
        f.close
        
#for checking in pymol to see if okay later
for_pymol = 'select'

In [5]:
# Use OpenMM to take out the residues?
top = pdb.getTopology()
pos = pdb.getPositions(True)


In [6]:
#Making lists of residue pairs for the structure /(believe me this was the easist way since 40 was missing)
left_side = [*range(0,11), *range(17,26)]
right_side = [*range(47,40,-1), *range(39,35,-1,), *range(34,25,-1)]

#Check to make sure your lists are what you need for your particular structure
print(left_side)
print(right_side)

#check that they're the same length
print(len(left_side))
print(len(right_side))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[47, 46, 45, 44, 43, 42, 41, 39, 38, 37, 36, 34, 33, 32, 31, 30, 29, 28, 27, 26]
20
20


In [7]:
for r in (0,1):
    
    for t in range(len(left_side)):
        L = left_side[t]
        R = right_side[t]
   
        #Check for the hyrodegens on the leftside, then the rightside
        if r == 0:
            A = L
            B = R
        else:
            B = L
            A = R
        
        resA = list(top.residues())[A]
        atom_idsA = []
        for atom in resA.atoms():
            atom_idsA.append(atom.id)

        resB = list(top.residues())[B]
        atom_idsB = []
        for atom in resB.atoms():
            atom_idsB.append(atom.id)
      
    
    

        first_H = [] 
        n_list = []
        for atom in resA.atoms():
            if atom.element.symbol == 'H':
                n_list.append(atom.id)
                first_H.append(pos[atom.index])
    
        non_H = []
        m_list = []
        for atom in resB.atoms():
            if atom.element.symbol != 'H':
                m_list.append(atom.id)
                non_H.append(pos[atom.index])
        non_Hv=np.array(non_H)

        

        for i in range(len(first_H)):

            first_Hv = np.array(first_H[i])


            dist = np.linalg.norm(non_Hv-first_Hv, axis=1)
            mini = np.min(dist)
            if mini < .25:
                j = np.argmin(dist)
                


                newl = 'd{:d}: DISTANCE ATOMS={:d},{:d}'.format(k,int(n_list[i]), int(m_list[j]))
                p1newl = ' id {:d} or id {:d} or'.format(int(n_list[i]), int(m_list[j])) #for pymol easy access
                for_pymol += p1newl    

                with open('plumed2.dat', 'a') as f:
                        f.write('\n')
                        f.write(newl)
                        k = k+1


In [8]:

print(for_pymol)

#Should use this to inspect the atoms it collected

# There will be weirdness around the loop at the top; may need to remove them from the plumed file
# by hand based on what it looks like

# I have been doing this by turning the whole structure white; using the selection command to turn all
# the selected atoms a color and the visually inspecting them

# I chose not to automate this removal because when used for other comformations (other than B), it will be 
# different each time

select id 82 or id 1453 or id 116 or id 1423 or id 150 or id 1385 or id 181 or id 1351 or id 212 or id 1321 or id 245 or id 1256 or id 276 or id 1226 or id 306 or id 1193 or id 339 or id 1163 or id 591 or id 1068 or id 621 or id 1038 or id 624 or id 1042 or id 654 or id 1004 or id 685 or id 974 or id 715 or id 943 or id 718 or id 945 or id 749 or id 906 or id 780 or id 872 or id 818 or id 826 or id 822 or id 837 or id 1451 or id 80 or id 1421 or id 114 or id 1387 or id 151 or id 1390 or id 153 or id 1353 or id 182 or id 1356 or id 184 or id 1323 or id 213 or id 1258 or id 246 or id 1261 or id 248 or id 1225 or id 274 or id 1195 or id 307 or id 1165 or id 341 or id 1070 or id 593 or id 1040 or id 622 or id 1006 or id 655 or id 1009 or id 657 or id 973 or id 683 or id 942 or id 713 or id 908 or id 750 or id 911 or id 752 or id 874 or id 783 or id 877 or id 783 or id 838 or id 821 or
